In [1]:
import os
import sys
sys.path.append("..")

from Volttron_interface_os_c_1a_zone_temp_adj_inc_ratch import (
    VolttronControls,
    VolttronControlFunctions,
    VolttronInterface
)

# Select control and test case identifier 
control = 'os_c_1a_zone_temp_adj_inc_ratch' 

# os_c_1a_zone_temp_adj_inc_ratch
# os_c_1b_zone_temp_adj_perform_ratch
# os_c_2a_zone_pre_heat_cool_sim
# os_c_2b_zone_pre_heat_cool_com
# os_c_1a2a_zone_temp_adj_inc_ratch_pre_heat_cool_sim
# os_c_1a2b_zone_temp_adj_inc_ratch_pre_heat_cool_com
# os_c_1b2a_zone_temp_adj_perform_ratch_pre_heat_cool_sim
# os_c_1b2b_zone_temp_adj_perform_ratch_pre_heat_cool_com
# os_c_3_chi_water_temp_reset

testcase = 'stanford_tes_berg' 
graph_path = os.path.join(os.getcwd(), f'{testcase}', f'{testcase}.ttl')

In [2]:
from utility_functions.get_price_threshold import get_price_threshold

class TestCaseInterface(VolttronInterface):
    def __init__(self, controls, config_path):
        super().__init__(controls, config_path)
    
    def get_price_threshold(self, *args):
        return get_price_threshold(*args)
    
def create_application(config_path) -> VolttronInterface:
    '''takes a config file and returns an instantiated TestCaseInterface
    
    Parameters
    ----------
        config_path : str
            the path to a configuration file for instantiating a bestest air shed application 
    Returns
    -------
        application : TestCaseInterface
            returns an instantiated bestest air interface with shedding controls
'''
    
    functions = VolttronControlFunctions()
    controls = VolttronControls(functions)
    
    # Instantiate the TestCaseInterface with the controls, config_path, and hvac_signal_function
    application = TestCaseInterface(controls, config_path)
    return application


In [3]:
import os
config_path = os.path.join(os.getcwd(), f'{testcase}', f'config_{control}.yml')

app = create_application(config_path)

# print of the control's SPARQL result

(rdflib.term.URIRef('urn:bldg/tes_berg_hvacZone'), rdflib.term.Literal('sensibo/tes_berg/ptac/temperature'), None, None, rdflib.term.Literal('occ_tes_berg'), None, None, rdflib.term.Literal('sensibo/tes_berg/ptac/targetTemperature'), None, None, None, None)
(rdflib.term.URIRef('urn:bldg/exp_berg_hvacZone'), rdflib.term.Literal('sensibo/exp_berg/ptac/temperature'), None, None, rdflib.term.Literal('occ_exp_berg'), None, None, rdflib.term.Literal('sensibo/exp_berg/ptac/targetTemperature'), None, None, None, None)


In [4]:
current_time = 0
operation_mode = 'heat'

y = {
        'sensibo/tes_berg/ptac/temperature': 19,
        'sensibo/tes_berg/ptac/targetTemperature': 16,
        'sensibo/exp_berg/ptac/temperature': 19,
        'sensibo/exp_berg/ptac/targetTemperature': 16  
    }

output = app.control_agent(current_time, operation_mode, y)

print(f"Output at time {current_time}: {output}")

0
{'sensibo/tes_berg/ptac/temperature': 19, 'sensibo/tes_berg/ptac/targetTemperature': 16, 'sensibo/exp_berg/ptac/temperature': 19, 'sensibo/exp_berg/ptac/targetTemperature': 16}
{}
price_threshold_value 0.2666
range(0, 2) ['urn:bldg/tes_berg_hvacZone', 'urn:bldg/exp_berg_hvacZone'] ['sensibo/tes_berg/ptac/temperature', 'sensibo/exp_berg/ptac/temperature'] ['sensibo/tes_berg/ptac/targetTemperature', 'sensibo/exp_berg/ptac/targetTemperature'] [] [] [] [] ['occ_tes_berg', 'occ_exp_berg'] [] [] [] None None None
heat
TSetHeaZon 16 TSetHeaZon_baseline [16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 16.0, 16.0, 16.0, 16.0]
TSetCooZon None TSetCooZon_baseline []
TZon 19
occupancy_schedule [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
price_schedule [0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2666, 0.2666, 0.4, 0.6, 0.8, 0.9, 0.9666, 0.2666, 0.2666, 0.2383, 0.2383, 0.2383

In [5]:
current_time = 1
operation_mode = 'heat'

# Initialize previous values for both target temperatures
prev_target_temp_tes_berg = None
prev_target_temp_exp_berg = None

for current_time in range(1, 24): 
    # Get the target temperature values, default to the previous value if not available
    target_temp_tes_berg = output.get('sensibo/tes_berg/ptac/targetTemperature', prev_target_temp_tes_berg)
    target_temp_exp_berg = output.get('sensibo/exp_berg/ptac/targetTemperature', prev_target_temp_exp_berg)

    y = {
        'sensibo/tes_berg/ptac/temperature': 19,
        'sensibo/tes_berg/ptac/targetTemperature': target_temp_tes_berg,
        'sensibo/exp_berg/ptac/temperature': 19,
        'sensibo/exp_berg/ptac/targetTemperature': target_temp_exp_berg
    }

    output = app.control_agent(current_time, operation_mode, y)

    print(f"Output at time {current_time}: {output}")

    # Update previous values for the next iteration
    prev_target_temp_tes_berg = target_temp_tes_berg
    prev_target_temp_exp_berg = target_temp_exp_berg


1
{'sensibo/tes_berg/ptac/temperature': 19, 'sensibo/tes_berg/ptac/targetTemperature': 16.0, 'sensibo/exp_berg/ptac/temperature': 19, 'sensibo/exp_berg/ptac/targetTemperature': 16.0}
{0: 0, 1: 0}
price_threshold_value 0.2666
range(0, 2) ['urn:bldg/tes_berg_hvacZone', 'urn:bldg/exp_berg_hvacZone'] ['sensibo/tes_berg/ptac/temperature', 'sensibo/exp_berg/ptac/temperature'] ['sensibo/tes_berg/ptac/targetTemperature', 'sensibo/exp_berg/ptac/targetTemperature'] [] [] [] [] ['occ_tes_berg', 'occ_exp_berg'] [] [] [] None None None
heat
TSetHeaZon 16.0 TSetHeaZon_baseline [16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 16.0, 16.0, 16.0, 16.0, 16.0]
TSetCooZon None TSetCooZon_baseline []
TZon 19
occupancy_schedule [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
price_schedule [0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2666, 0.2666, 0.4, 0.6, 0.8, 0.9, 0.9666, 0.2666, 0.2666, 0.2383, 0.2383